# IMDB 영화평 감성분석

- Pipeline
- TfidfVectorizer + LogisticRegression

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
up = files.upload()

Saving labeledTrainData.tsv to labeledTrainData.tsv


In [ ]:
df = pd.read_csv("labeledTrainData.tsv", sep = "\t")
df.head(3)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...


In [ ]:
import warnings
warnings.filterwarnings("ignore")
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace("<br />", " ")# 구둣점, 숫자 제거 - 영문자가 아닌 글자는 공백으로 변환
# 데이터프레임의 str메소드는 정규표현식을 지원함
df.review = df.review.str.replace("[^A-Za-z]", " ").str.strip()
df.review[0][:1000]

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

- Train/Test datasets분리

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify = df.sentiment, random_state =2022
)
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

- Pipeline : TfidfVectorizer + Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
tvect = TfidfVectorizer(ngram_range=(1,2), stop_words="english")
lrc = LogisticRegression(random_state = 2022)

In [ ]:
pipeline = Pipeline([
      ("TVECT",tvect), ("LR", lrc)
])

In [ ]:
# 학습
%time pipeline.fit(X_train, y_train)

CPU times: user 32.7 s, sys: 13.4 s, total: 46.1 s
Wall time: 34.3 s


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('LR', LogisticRegression(random_state=2022))])

In [ ]:
pipeline.score(X_test,y_test)

0.87472

- 최적 파라메터 찾기

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    "TVECT__max_df" : [100, 500],
    "LR__C" : [1, 10]
}

In [ ]:
grid_pipe = GridSearchCV(pipeline, params, scoring ="accuracy", cv = 3, n_jobs = -1) # -1 을 주면 병행처리를 할 수 있다
%time grid_pipe.fit(X_train,y_train)

CPU times: user 39.3 s, sys: 19.3 s, total: 58.6 s
Wall time: 2min 52s


{'LR__C': 10, 'TVECT__max_df': 500}

In [ ]:
grid_pipe.best_params_


{'LR__C': 10, 'TVECT__max_df': 500}

In [ ]:
grid_pipe.best_estimator_.score(X_test,y_test)

0.87552

In [ ]:
import joblib

In [ ]:
joblib.dump(grid_pipe.best_estimator_, "imdb_tvect_lr.pkl")

['imdb_tvect_lr.pkl']

- 실제 데이터 적용

In [ ]:
review = ""

In [ ]:
# 텍스트 전처리
import re
review = re.sub('[^A-Za-z]'," ",review).strip()
best_estimater = joblib.load("imdb_tvect_lr.pkl")
best_estimater.predict([review])

array([0])